# Module Imports

In [22]:
## IMPORT MODULES
import pandas as pd
import numpy as np
from sklearn import linear_model
from scipy.interpolate import griddata
import scipy.odr


In [23]:
## IMPORT PLOTTING MODULES
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib as mpl
from matplotlib import cm
%matplotlib widget

# Dataframe

In [24]:
## FULL DATAFRAME
df_raw = pd.read_csv("WS_2-Policy_Summary.csv")

## DROP EXTRANEOUS DATAPOINTS
# df_raw = df_raw.drop(df_raw[(df_raw['landing_rate']<=0.15) & (df_raw['vz_d']>=2.5)].index)
# df_raw = df_raw.drop(df_raw[(df_raw['landing_rate']<=0.15) & (df_raw['vx_d']>=1.0)].index)
# df_raw = df_raw.dropna()
df_raw = df_raw.drop(df_raw[df_raw['My_d']<-7.7].index)




## GROUP RAW DATA BY IC AND FILTER BY MAX LANDING RATE
df_raw_max = df_raw.groupby(['vz_d','vx_d']).agg({'landing_rate':"max"}).reset_index()
idx = df_raw.groupby(['vz_d','vx_d'])['landing_rate'].transform(max) == df_raw['landing_rate']
df_max = df_raw[idx].reset_index()


## CLEAN RAW DATA
df_max['My_d'] = df_max['My_d'].apply(lambda x: np.abs(x)) # Convert My to magnitude
df_max['My_d'] = df_max['My_d'].apply(lambda x: 7.7 if x>7.7 else x) # Cap My_d to max possible moment




In [26]:
# TEMPLATE FOR LINEAR REGRESSION
from sklearn.linear_model import LinearRegression

def linreg(X ,Y):
    # perform multivariate or univariate
    # X = [var1 var2]
    # y = var3
    reg = LinearRegression(normalize=True)
    reg.fit(X,Y)

    intercept = reg.intercept_ 
    coef = reg.coef_
    
    R2 = reg.score(X,Y) # built in function for r^2
    
    ypred = reg.predict(X) # plot x vs. ypred for line or plane
    
    return ypred,intercept,coef,R2


# Landing Rate Data

In [27]:
## LANDING RATE DATA (RAW)
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# DEFINE VARIABLES
X = df_max['vx_d']
Y = df_max['vz_d']
Z = df_max['landing_rate']


# CREATE PLOTS AND COLOR BAR
ax.scatter(X,Y,Z)


# PLOT LIMITS AND INFO
ax.set_zlim(0,1)

ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('Landing Rate (%)')
ax.set_title('Optimal Policy Landing Rate')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
## AVG LANDING RATE SURFACE
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# DEFINE VARIABLES
X = df_max['vx_d']
Y = df_max['vz_d']
Z = df_max['landing_rate']

# DEFINE COLOR FORMATS AND LIMITS
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=1)

# CREATE PLOTS AND COLOR BAR
ax.plot_trisurf(X,Y,Z,cmap = cmap,norm=norm,edgecolors='grey',linewidth=0.25)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate (%)")


# PLOT LIMITS AND INFO
ax.set_zlim(0,1)

ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('Landing Rate (%)')
ax.set_title('Landing Rate (Raw Data)')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: x and y arrays must have a length of at least 3

In [15]:
df_temp = df_max.query(f"vx_d<={0.0} & vz_d<={2.5}")
df_temp

,index,vz_d,vx_d,trial_num,landing_rate,RREV_threshold,G1,G2,RREV_sig,G1_sig,G2_sig,RREV_trigger,OF_y,My_d,impact_eul,impact_tdelta,alpha_mu,alpha_sigma,mu_ini,sigma_ini
30,313,2.25,0.0,0,0.566667,4.50,7.13,5.59,0.13,0.04,0.07,4.856765,0.001706,7.157882,-36.362742807358394,0.20023529411764518,[0. 0.],[0. 0.],[1.74 6.38 5.85],[2. 2. 2.]
44,409,2.50,0.0,2,0.500000,4.28,5.86,3.82,0.05,0.09,0.01,4.595667,0.001733,5.825067,-157.1441296858511,0.24346666666664685,[0. 0.],[0. 0.],[6.92 5.37 4.31],[2. 2. 2.]


In [29]:
## AVG LANDING RATE SMOOTHED SURFACE
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# DEFINE VARIABLES
X = df_max['vx_d']
Y = df_max['vz_d']
Z = df_max['landing_rate']

# SOMETHING ABOUT DEFINING A GRID
xi = np.linspace(X.min(),X.max(),(len(Z)//3))
yi = np.linspace(Y.min(),Y.max(),(len(Z)//3))
zi = griddata((X, Y), Z, (xi[None,:], yi[:,None]), method='linear')
xig, yig = np.meshgrid(xi, yi)

# DEFINE COLOR FORMAT AND LIMITS
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=1)

# CREATE PLOTS AND COLOR BAR
surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm,edgecolors='grey',linewidth=0.25)
# surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm)
# surf = ax.contour(xig, yig, zi,cmap=cmap,norm=norm)

fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate (%)")

# PLOT LIMITS AND INFO
ax.set_zlim(0,1)

ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('Landing Rate (%)')
ax.set_title('Landing Rate (Raw Data)')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: No points given

# RREV vs IC

In [17]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df_max.query(f"landing_rate>={0.4}")




# LINEAR REGRESSION
X_reg = df_temp[['vx_d','vz_d']]
Y_reg = df_temp['RREV_threshold']

ypred, intercept,coef,R2 = linreg(X_reg,Y_reg)
print(f"Equation: RREV_reg = {intercept:.2f} + {coef[0]:.2f}*vx_d + {coef[1]:.2f}*vz_d \n")
print(f"R^2 = {R2:.3f}")


# DEFINE VARIABLES
X = df_temp['vx_d']
Y = df_temp['vz_d']
Z = df_temp['RREV_threshold']
C = df_temp['RREV_threshold']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=3.5,vmax=5.5)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm,zorder=1)
ax.plot_trisurf(X,Y,ypred,alpha=0.5,label='Linear_Regression',zorder=2)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="RREV_threshold (1/s)")



# PLOT LIMITS AND INFO
ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.5)
ax.set_zlim(3,6)

ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('RREV_threshold (1/s)')
ax.set_title('RREV_thr vs IC - (Optimal Data)')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Equation: RREV_reg = 5.51 + 0.00*vx_d + -0.35*vz_d 

R^2 = 0.590


In [18]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df_max.query(f"landing_rate>={0.4}")


# DEFINE VARIABLES
X = df_temp['vx_d']
Y = df_temp['vz_d']
Z = df_temp['RREV_threshold']

# SOMETHING ABOUT DEFINING A GRID
xi = np.linspace(X.min(),X.max(),(len(Z)//10))
yi = np.linspace(Y.min(),Y.max(),(len(Z)//10))
zi = griddata((X, Y), Z, (xi[None,:], yi[:,None]), method='linear')
xig, yig = np.meshgrid(xi, yi)

# DEFINE COLOR FORMAT AND LIMITS
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=3.5,vmax=5.5)

# CREATE PLOTS AND COLOR BAR
surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm,edgecolors='grey',linewidth=0.25)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="RREV_threshold")

# PLOT LIMITS AND INFO
ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.5)
ax.set_zlim(3,6)

ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('RREV_threshold (1/s)')

ax.set_title('Optimal RREV_thr vs IC')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Rotation Time Data

In [19]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df_max.query(f"landing_rate>={0.4}")

# DEFINE VARIABLES
X = df_temp['RREV_threshold']
Y = df_temp['OF_y']
Z = df_temp['impact_tdelta']
C = df_temp['landing_rate']


# CREATE PLOTS AND COLOR BAR
ax.scatter(X,Y,Z)

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate %)")



# PLOT LIMITS AND INFO
ax.set_xlabel('RREV (1/s)')
ax.set_ylabel('OF_y (rad/s)')
ax.set_zlabel('Delta_t (s)')

ax.set_title('Time Rotating vs IC (Optimal Data)')

ax.set_zlim(0,0.5)




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# My_d vs IC Data

In [20]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df_max.query(f"landing_rate>={0.4}")

# DEFINE VARIABLES
X = df_temp['vx_d']
Y = df_temp['vz_d']
Z = df_temp['My_d']
C = df_temp['landing_rate']


# CREATE PLOTS AND COLOR BAR
ax.scatter(X,Y,Z)

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate %)")



# PLOT LIMITS AND INFO
ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('My_d (N*mm)')

ax.set_title('My_d vs IC (Optimal Data)')

ax.set_zlim(0,10)




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Impact Angle Data

In [21]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df_max.query(f"landing_rate>={0.4}")


# DEFINE VARIABLES
X = df_temp['vx_d']
Y = df_temp['vz_d']
Z = -df_temp['impact_eul']
C = df_temp['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")



# PLOT LIMITS AND INFO
ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('Impact Angle (deg)')

ax.set_title('Impact Angle vs IC (Optimal Data)')

ax.set_zlim(60,180)
ax.set_zticks([60,90,120,150,180,210])



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

TypeError: bad operand type for unary -: 'str'

In [14]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df_max.query(f"landing_rate>={0.4}")


# DEFINE VARIABLES
X = df_temp['vx_d']
Y = df_temp['vz_d']
Z = -df_temp['impact_eul']
C = df_temp['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=60,vmax=180)
ax.plot_trisurf(X,Y,Z,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate",orientation="horizontal")



# PLOT LIMITS AND INFO
ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('Impact Angle (deg)')

ax.set_title('Impact Angle vs IC (Optimal Data)')

ax.set_zlim(60,180)
ax.set_zticks([60,90,120,150,180,210])



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
## AVG LANDING RATE SMOOTHED SURFACE
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df_max.query(f"landing_rate>={0.4}")

# DEFINE VARIABLES
X = df_temp['vx_d']
Y = df_temp['vz_d']
Z = -df_temp['impact_eul']

# SOMETHING ABOUT DEFINING A GRID
xi = np.linspace(X.min(),X.max(),(len(Z)//10))
yi = np.linspace(Y.min(),Y.max(),(len(Z)//10))
zi = griddata((X, Y), Z, (xi[None,:], yi[:,None]), method='linear')
xig, yig = np.meshgrid(xi, yi)

# DEFINE COLOR FORMAT AND LIMITS
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=60,vmax=180)

# CREATE PLOTS AND COLOR BAR
surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm,edgecolors='grey',linewidth=0.25)
# surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm)
# surf = ax.contour(xig, yig, zi,cmap=cmap,norm=norm)

fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Impact Angle (deg)")

# PLOT LIMITS AND INFO
ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('Impact Angle (deg)')

ax.set_title('Impact Angle vs IC (Optimal Data)')

ax.set_zlim(60,180)
ax.set_zticks([60,90,120,150,180,210])


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Moment vs RREV

In [16]:
fig = plt.figure()
ax = fig.add_subplot(111)

# CLEAN DATA
df_temp = df_max.query(f"landing_rate>={0.4}")

# DEFINE VARIABLES
X = df_temp['RREV_threshold']
Y = df_temp['My_d']
C = df_temp['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.Greys
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate (%)")



# PLOT LIMITS AND INFO
ax.set_xlabel('RREV_threshold (1/s)')
ax.set_ylabel('My_d (N*mm)')
ax.set_title('Policy: My_d vs RREV_thr')


ax.grid()
ax.set_xlim(2,7)
ax.set_ylim(0,10)
ax.hlines(7.77,2,7)
ax.text(2.05,8.1,'Motors Maxed Out \n(My_d = 7.7 N*mm)')




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
fig = plt.figure()
ax = fig.add_subplot(111)

# CLEAN DATA
df_temp = df_max.query(f"landing_rate>={0.4}")

# DEFINE VARIABLES
X = df_temp['RREV_threshold']
Y = df_temp['My_d']
C = df_temp['vz_d']

# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=1.5,vmax=4)
ax.scatter(X,Y,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Z-Velocity (m/s)")



# PLOT LIMITS AND INFO
ax.set_xlabel('RREV_threshold (1/s)')
ax.set_ylabel('My_d (N*mm)')
ax.set_title('Policy: My_d vs RREV_thr')




ax.grid()
ax.set_xlim(2,7)
ax.set_ylim(0,10)
ax.hlines(7.77,2,7)
ax.text(2.05,8.1,'Motors Maxed Out \n(My_d = 7.7 N*mm)')




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# POLICY FITTING

## RAW POLICY RELATION

In [18]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df_max.query(f"landing_rate>={0.6}")



# DEFINE VARIABLES
X = df_temp['RREV_threshold']
Y = df_temp['OF_y']
Z = df_temp['My_d']
C = df_temp['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")



# PLOT LIMITS AND INFO
ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV (1/s)')
ax.set_ylabel('OF_y (rad/s)')
ax.set_zlabel('My_d (N*mm)')

ax.set_title('Raw Policy Relation')



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# LINEAR MODEL FIT

## Regression with y-based loss

In [19]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_reg = df_max.query(f"landing_rate>={0.6}")


# LINEAR REGRESSION
X_reg = df_reg[['RREV_threshold','OF_y']]
Y_reg = df_reg['My_d']

ypred, intercept,coef,R2 = linreg(X_reg,Y_reg)
print(f"Equation: My_d = {intercept:.2f} + {coef[0]:.2f}*RREV + {coef[1]:.2f}*OF_y \n")
print(f"R^2 = {R2:.3f}")



# DEFINE VARIABLES
X = df_reg['RREV_threshold']
Y = df_reg['OF_y']
Z = df_reg['My_d']
C = df_reg['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.plot_trisurf(X_reg['RREV_threshold'],X_reg['OF_y'],ypred,alpha=0.75,label='Linear_Regression',zorder=1) # Surface plot
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm) # Scatter plot
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")



# PLOT LIMITS AND INFO
ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV (1/s)')
ax.set_ylabel('OF_y (rad/s)')
ax.set_zlabel('My_d (N*mm)')

ax.set_title('Optimal Policy Relation \n(Z-Based Error)')



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Equation: My_d = -5.21 + 2.32*RREV + 0.40*OF_y 

R^2 = 0.173


In [24]:
df_cov = df_reg[['RREV_threshold','OF_y','My_d']]
df_cov.cov()

,RREV_threshold,OF_y,My_d
RREV_threshold,0.128934,-0.366187,0.153750
OF_y,-0.366187,3.202645,0.418622
My_d,0.153750,0.418622,3.016097


In [20]:
df_reg.query(f"My_d<={3.5}").sort_values('OF_y')

,index,vz_d,vx_d,trial_num,landing_rate,RREV_threshold,G1,G2,RREV_sig,G1_sig,G2_sig,RREV_trigger,OF_y,My_d,impact_eul,impact_tdelta,alpha_mu,alpha_sigma,mu_ini,sigma_ini
6,87,1.50,2.25,3,0.964286,5.04,3.09,1.66,0.00,0.03,0.04,5.700370,-6.846444,3.089370,-79.882310,0.199778,[0. 0.],[0. 0.],[5.53 6.58 3.12],[2. 2. 2.]
29,308,2.00,2.75,3,1.000000,4.84,2.28,2.42,0.02,0.09,0.00,5.231533,-6.063600,2.301367,-92.828947,0.249867,[0. 0.],[0. 0.],[4.28 3.44 3.41],[2. 2. 2.]
15,178,1.75,2.00,5,1.000000,4.75,8.38,1.96,0.09,1.50,0.46,5.603286,-5.736214,3.404357,-91.498347,0.207500,[0. 0.],[0. 0.],[4.27 2.1 1.26],[2. 2. 2.]
27,293,2.00,2.25,3,0.956522,4.79,7.16,3.17,0.06,0.10,0.25,5.497864,-5.611909,2.168455,-90.439828,0.250091,[0. 0.],[0. 0.],[3. 6.37 2.39],[2. 2. 2.]
56,494,2.50,2.75,5,0.933333,4.87,9.44,4.15,0.02,0.52,0.14,5.339429,-5.429357,2.949464,-105.826224,0.229857,[0. 0.],[0. 0.],[3.46 4.64 3.83],[2. 2. 2.]
14,169,1.75,1.75,3,0.958333,4.95,6.27,2.00,0.01,0.03,0.13,5.816348,-5.210217,2.618348,-80.102154,0.220130,[0. 0.],[0. 0.],[2.63 5.71 4.73],[2. 2. 2.]
39,379,2.25,2.25,0,0.866667,4.94,4.68,0.50,0.02,0.11,0.11,5.548846,-5.084692,2.315192,-84.448864,0.236192,[0. 0.],[0. 0.],[6.28 5.01 2.19],[2. 2. 2.]
4,75,1.50,1.75,3,0.875000,4.86,3.32,1.60,0.00,0.00,0.00,5.288762,-4.940667,3.321476,-102.548794,0.226952,[0. 0.],[0. 0.],[6.57 4.7 4.59],[2. 2. 2.]
3,64,1.50,1.50,0,0.600000,5.35,2.56,1.49,0.03,0.10,0.20,5.778222,-4.626611,2.557667,-74.914026,0.210556,[0. 0.],[0. 0.],[5.17 2.78 1.13],[2. 2. 2.]
26,283,2.00,2.00,3,1.000000,4.54,2.26,4.90,0.01,0.05,0.63,5.213650,-4.393100,2.252100,-93.922152,0.248400,[0. 0.],[0. 0.],[1.94 4.35 1.46],[2. 2. 2.]


## Regression with x-based loss


In [21]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_reg = df_max.query(f"landing_rate>={0.6}")


# LINEAR REGRESSION
X_reg = df_reg[['My_d','OF_y']]
Y_reg = df_reg['RREV_threshold']

ypred, intercept,coef,R2 = linreg(X_reg,Y_reg)
print(f"Equation: RREV_thr = {intercept:.2f} + {coef[0]:.2f}*My_d + {coef[1]:.2f}*OF_y \n")
# print(f"Equation: My_d = ((RREV_thr - {intercept:.2f}) - {coef[1]:.2f}*OF_y)/{coef[0]:.2f})")
print(f"R^2 = {R2:.3f}")



# DEFINE VARIABLES
X = df_reg['My_d']
Y = df_reg['OF_y']
Z = df_reg['RREV_threshold']
C = df_reg['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.plot_trisurf(X_reg['My_d'],X_reg['OF_y'],ypred,alpha=0.75,label='Linear_Regression',zorder=1) # Surface plot
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm) # Scatter plot
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate (%)")



# PLOT LIMITS AND INFO
ax.set_xlim(10,0)
ax.set_ylim(0,-10)
ax.set_zlim(3,6)



ax.set_xlabel('My_d (N*mm)')
ax.set_ylabel('OF_y (rad/s)')
ax.set_zlabel('RREV (1/s)')

ax.set_title('Optimal Policy Relation \n(X-Based Error)')



plt.show()




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Equation: RREV_thr = 3.88 + 0.07*My_d + -0.12*OF_y 

R^2 = 0.431


# Regression w/ Orthogonal Distance

In [22]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_reg = df_max.query(f"landing_rate>={0.6}")


# LINEAR REGRESSION
def linfit(beta, x):
    return beta[0]*x[0] + beta[1]*x[1] + beta[2] 

x1 = df_reg['RREV_threshold']
x2 = df_reg['OF_y']
y = df_reg['My_d']
x = np.row_stack( (x1, x2) ) #odr doesn't seem to work with column_stack

linmod = scipy.odr.Model(linfit)
data = scipy.odr.Data(x, y)
odrfit = scipy.odr.ODR(data, linmod, beta0=[1., 1., 1.])
odrres = odrfit.run()
odrres.pprint()


y_reg = odrres.beta[0]*x1 + odrres.beta[1]*x2 + odrres.beta[2]

print(f"Equation: My_d = {odrres.beta[2]:.2f} + {odrres.beta[0]:.2f}*RREV + {odrres.beta[1]:.2f}*OF_y \n")
# print(f"R^2 = {R2:.3f}")



# DEFINE VARIABLES
X = df_max['RREV_threshold']
Y = df_max['OF_y']
Z = df_max['My_d']
C = df_max['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.plot_trisurf(x1,x2,y_reg,alpha=0.75,label='Linear_Regression',zorder=1) # Surface plot
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm) # Scatter plot
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate (%)")



# PLOT LIMITS AND INFO
ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV (1/s)')
ax.set_ylabel('OF_y (rad/s)')
ax.set_zlabel('My_d (N*mm)')

ax.set_title('Optimal Policy Relation \n(Orthogonal Regression)')



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Beta: [ 14.24775773   1.80019726 -54.95025462]
Beta Std Error: [ 3.3874206   0.45801538 14.15137079]
Beta Covariance: [[ 156.13758091   18.29283331 -651.16969919]
 [  18.29283331    2.8544952   -74.05045078]
 [-651.16969919  -74.05045078 2724.99819251]]
Residual Variance: 0.07349043227476758
Inverse Condition #: 0.004802836564341187
Reason(s) for Halting:
  Sum of squares convergence
Equation: My_d = -54.95 + 14.25*RREV + 1.80*OF_y 



In [23]:
import numpy as np
import scipy.odr

n = 1000
t = np.linspace(0, 1, n)

def linfit(beta, x):
    return beta[0]*x[0] + beta[1]*x[1] + beta[2] #notice changed indices for x

x1 = 2.5*np.sin(2*np.pi*6*t)+4
x2 = 0.5*np.sin(2*np.pi*7*t + np.pi/3)+2

x = np.row_stack( (x1, x2) ) #odr doesn't seem to work with column_stack

e = 0.25*np.random.randn(n)
y = 3*x[0] + 4*x[1] + 5 + e #indices changed

linmod = scipy.odr.Model(linfit)
data = scipy.odr.Data(x, y)
odrfit = scipy.odr.ODR(data, linmod, beta0=[1., 1., 1.])
odrres = odrfit.run()
odrres.pprint()


Beta: [3.00757658 4.0363251  4.89358429]
Beta Std Error: [0.00442904 0.02225373 0.04857541]
Beta Covariance: [[ 8.43859074e-03  7.28986530e-05 -3.39001941e-02]
 [ 7.28986530e-05  2.13037410e-01 -4.26458700e-01]
 [-3.39001941e-02 -4.26458700e-01  1.01504036e+00]]
Residual Variance: 0.002324607792610536
Inverse Condition #: 0.03035728824479211
Reason(s) for Halting:
  Sum of squares convergence


In [25]:
odrres.beta

array([3.00757658, 4.0363251 , 4.89358429])

In [26]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
y_reg = odrres.beta[0]*x1 + odrres.beta[1]*x2 + odrres.beta[2]

ax.scatter(x1,x2,y)
ax.plot_trisurf(x1,x2,y_reg)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …